<a href="https://colab.research.google.com/github/abalckpie/professorahn/blob/main/%EB%B0%98%EB%B3%B5%EC%B8%A1%EC%A0%95_%EB%B3%80%EB%9F%89%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 필요한 라이브러리 설치

In [42]:
!pip install pingouin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 3.9 MB/s eta 0:00:00


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM
import scikit_posthocs as sp
import pingouin as pg
import scipy.stats as stats

2. 데이터 준비

In [17]:
csv_test = pd.read_csv('/content/2022_2022년 지능정보사회 이용자 패널 조사_Data(가중치포함).csv')
data = csv_test.copy()
columns_of_interest = ['Age_group'] + [f'Q30_{i}' for i in range(1, 25)]
subset_data = data[columns_of_interest]

3. 그룹별 평균 계산

In [18]:
subset_data['생활활용'] = subset_data[[f'Q30_{i}' for i in range(1, 7)]].mean(axis=1)
subset_data['권리보호'] = subset_data[[f'Q30_{i}' for i in range(7, 12)]].mean(axis=1)
subset_data['비판적 이해'] = subset_data[[f'Q30_{i}' for i in range(12, 16)]].mean(axis=1)
subset_data['생산과 공유'] = subset_data[[f'Q30_{i}' for i in range(16, 19)]].mean(axis=1)
subset_data['사회 참여'] = subset_data[[f'Q30_{i}' for i in range(19, 22)]].mean(axis=1)
subset_data['보안'] = subset_data[[f'Q30_{i}' for i in range(22, 25)]].mean(axis=1)

<ipython-input-18-50d5c1a80b22>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['생활활용'] = subset_data[[f'Q30_{i}' for i in range(1, 7)]].mean(axis=1)
<ipython-input-18-50d5c1a80b22>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['권리보호'] = subset_data[[f'Q30_{i}' for i in range(7, 12)]].mean(axis=1)
<ipython-input-18-50d5c1a80b22>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

4. 형식 변환(긴 형식, 범주형)

In [19]:
long_data = pd.melt(subset_data, id_vars=['Age_group'], value_vars=['생활활용', '권리보호', '비판적 이해', '생산과 공유', '사회 참여', '보안'],
                    var_name='Factor', value_name='Response')

In [20]:
long_data['Age_group'] = long_data['Age_group'].astype('category')

In [29]:
long_data.head()

,Age_group,Factor,Response
0,6,생활활용,3.000000
1,5,생활활용,3.333333
2,5,생활활용,5.000000
3,2,생활활용,2.666667
4,4,생활활용,4.000000


5. 데이터 수동 집계(피험자와 요인별 평균 계산)

In [21]:
agg_data = long_data.groupby(['Age_group', 'Factor']).mean().reset_index()

In [27]:
agg_data

,Age_group,Factor,Response
0,1,권리보호,3.163692
1,1,보안,3.447179
2,1,비판적 이해,3.504615
3,1,사회 참여,3.102564
4,1,생산과 공유,3.421538
5,1,생활활용,3.721538
6,2,권리보호,3.378253
7,2,보안,3.651218
8,2,비판적 이해,3.649955
9,2,사회 참여,3.426619


6. 반복측정 변량분석 수행 및 결과

In [22]:
aovrm = AnovaRM(agg_data, 'Response', 'Age_group', within=['Factor'])
res = aovrm.fit()

In [23]:
print(res)

               Anova
       F Value Num DF  Den DF Pr > F
------------------------------------
Factor 35.7822 5.0000 25.0000 0.0000



결과 해석 : F-value 값이 매우 크고(35.78) p-value가 0.0000으로 0.05보다 작기 때문에 6개 요인 간에 유의적인 차이가 있다.

사후검정 1 투키

In [28]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
tukey = pairwise_tukeyhsd(endog=long_data['Response'], groups=long_data['Factor'], alpha=0.05)
print(tukey)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
  권리보호     보안   0.0702 0.0002  0.0238  0.1166   True
  권리보호 비판적 이해   0.2334    0.0   0.187  0.2798   True
  권리보호  사회 참여   0.0346 0.2727 -0.0118   0.081  False
  권리보호 생산과 공유  -0.0097 0.9912 -0.0561  0.0367  False
  권리보호   생활활용   0.5765    0.0  0.5301  0.6229   True
    보안 비판적 이해   0.1631    0.0  0.1167  0.2095   True
    보안  사회 참여  -0.0356 0.2446  -0.082  0.0108  False
    보안 생산과 공유    -0.08    0.0 -0.1264 -0.0336   True
    보안   생활활용   0.5063    0.0  0.4599  0.5527   True
비판적 이해  사회 참여  -0.1987    0.0 -0.2451 -0.1523   True
비판적 이해 생산과 공유  -0.2431    0.0 -0.2895 -0.1967   True
비판적 이해   생활활용   0.3432    0.0  0.2968  0.3896   True
 사회 참여 생산과 공유  -0.0444 0.0703 -0.0908   0.002  False
 사회 참여   생활활용   0.5419    0.0  0.4955  0.5883   True
생산과 공유   생활활용   0.5862    0.0  0.5398  0.6326   True
----------------------------------------------

사후검정 2 던칸

In [34]:
!pip install statsmodels scikit-posthocs

In [44]:
#duncan_result = sp.posthoc_duncan(long_data, val_col='Response', group_col='Factor', alpha=0.05)
#print(duncan_result)
import pingouin as pg
duncan_result = pg.pairwise_tukey(dv='Response', between='Factor', data=long_data)
print(duncan_result)

         A       B   mean(A)   mean(B)      diff        se          T  \
0     권리보호      보안  3.094459  3.164683 -0.070224  0.016281  -4.313226   
1     권리보호  비판적 이해  3.094459  3.327817 -0.233358  0.016281 -14.333007   
2     권리보호   사회 참여  3.094459  3.129106 -0.034647  0.016281  -2.128061   
3     권리보호  생산과 공유  3.094459  3.084728  0.009731  0.016281   0.597685   
4     권리보호    생활활용  3.094459  3.670974 -0.576515  0.016281 -35.409952   
5       보안  비판적 이해  3.164683  3.327817 -0.163134  0.016281 -10.019781   
6       보안   사회 참여  3.164683  3.129106  0.035577  0.016281   2.185165   
7       보안  생산과 공유  3.164683  3.084728  0.079955  0.016281   4.910911   
8       보안    생활활용  3.164683  3.670974 -0.506291  0.016281 -31.096725   
9   비판적 이해   사회 참여  3.327817  3.129106  0.198711  0.016281  12.204946   
10  비판적 이해  생산과 공유  3.327817  3.084728  0.243089  0.016281  14.930692   
11  비판적 이해    생활활용  3.327817  3.670974 -0.343157  0.016281 -21.076944   
12   사회 참여  생산과 공유  3.129106  3.084728  0.044378  0

In [54]:
comparisons = duncan_result[['A', 'B', 'diff', 'se', 'T']].copy()
df_between = len(long_data) - len(long_data['Age_group'].unique())
comparisons['p-value'] = comparisons.apply(lambda row: stats.t.sf(abs(row['T']), df_between) * 2, axis=1)
print(comparisons)

         A       B      diff        se          T        p-value
0     권리보호      보안 -0.070224  0.016281  -4.313226   1.613668e-05
1     권리보호  비판적 이해 -0.233358  0.016281 -14.333007   1.890837e-46
2     권리보호   사회 참여 -0.034647  0.016281  -2.128061   3.333955e-02
3     권리보호  생산과 공유  0.009731  0.016281   0.597685   5.500547e-01
4     권리보호    생활활용 -0.576515  0.016281 -35.409952  1.758869e-269
5       보안  비판적 이해 -0.163134  0.016281 -10.019781   1.351027e-23
6       보안   사회 참여  0.035577  0.016281   2.185165   2.888392e-02
7       보안  생산과 공유  0.079955  0.016281   4.910911   9.109708e-07
8       보안    생활활용 -0.506291  0.016281 -31.096725  3.295267e-209
9   비판적 이해   사회 참여  0.198711  0.016281  12.204946   3.479948e-34
10  비판적 이해  생산과 공유  0.243089  0.016281  14.930692   3.063608e-50
11  비판적 이해    생활활용 -0.343157  0.016281 -21.076944   5.932974e-98
12   사회 참여  생산과 공유  0.044378  0.016281   2.725746   6.419098e-03
13   사회 참여    생활활용 -0.541868  0.016281 -33.281890  7.818783e-239
14  생산과 공유    생활활용 -0.586

In [46]:
results = []
comparisons = [
    ('권리보호', '보안'), ('권리보호', '비판적 이해'), ('권리보호', '사회 참여'),
    ('권리보호', '생산과 공유'), ('권리보호', '생활활용'), ('보안', '비판적 이해'),
    ('보안', '사회 참여'), ('보안', '생산과 공유'), ('보안', '생활활용'),
    ('비판적 이해', '사회 참여'), ('비판적 이해', '생산과 공유'), ('비판적 이해', '생활활용'),
    ('사회 참여', '생산과 공유'), ('사회 참여', '생활활용'), ('생산과 공유', '생활활용')
]

t_values = [
    -4.313226, -14.333007, -2.128061, 0.597685, -35.409952, -10.019781,
    2.185165, 4.910911, -31.096725, 12.204946, 14.930692, -21.076944,
    2.725746, -33.281890, -36.007636
]

In [51]:
df_between = len(data['Age_group']) - len(set(data['Age_group']))
for comparison, t_val in zip(comparisons, t_values):
    p_val = stats.t.sf(abs(t_val), df_between) * 2  # 양측 검정
    results.append((*comparison, t_val, p_val))

In [52]:
for result in results:
    print(f"Comparison: {result[0]} vs {result[1]}, t-value: {result[2]:.6f}, p-value: {result[3]:.6e}")


Comparison: 권리보호 vs 보안, t-value: -4.313226, p-value: 1.637734e-05
Comparison: 권리보호 vs 비판적 이해, t-value: -14.333007, p-value: 9.414932e-46
Comparison: 권리보호 vs 사회 참여, t-value: -2.128061, p-value: 3.337742e-02
Comparison: 권리보호 vs 생산과 공유, t-value: 0.597685, p-value: 5.500753e-01
Comparison: 권리보호 vs 생활활용, t-value: -35.409952, p-value: 4.850469e-247
Comparison: 보안 vs 비판적 이해, t-value: -10.019781, p-value: 2.001498e-23
Comparison: 보안 vs 사회 참여, t-value: 2.185165, p-value: 2.891980e-02
Comparison: 보안 vs 생산과 공유, t-value: 4.910911, p-value: 9.333960e-07
Comparison: 보안 vs 생활활용, t-value: -31.096725, p-value: 2.296715e-195
Comparison: 비판적 이해 vs 사회 참여, t-value: 12.204946, p-value: 8.173279e-34
Comparison: 비판적 이해 vs 생산과 공유, t-value: 14.930692, p-value: 2.016161e-49
Comparison: 비판적 이해 vs 생활활용, t-value: -21.076944, p-value: 8.145532e-95
Comparison: 사회 참여 vs 생산과 공유, t-value: 2.725746, p-value: 6.436433e-03
Comparison: 사회 참여 vs 생활활용, t-value: -33.281890, p-value: 5.391646e-221
Comparison: 생산과 공유 vs 생활활용, t-